In [408]:
import warnings

from time import time
import pandas as pd
from pandas.api.types import is_numeric_dtype
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
)
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVC

warnings.filterwarnings('ignore')

---
## 1
Загрузите предложенный вам датасет с помощью функции `sklearn.datasets.fetch_openml`.

Выведите текстовое описание загруженного датасета.

Обозначьте целевую переменную за `y`, а остальные данные за `X`.

In [409]:
# Загружем датасет
dataset = fetch_openml('students_scores')

In [410]:
# Выводим текстовое описание датасета
print(dataset.DESCR)

This is a dataset about the actors and actresses that have
voice characters in Disney movies

Downloaded from openml.org.


In [411]:
# Обознчим переменные
X = dataset.data
y = dataset.target

In [412]:
X.sample(5)

,race.ethnicity,parental.level.of.education,lunch,test.preparation.course,math.score,reading.score,writing.score
782,group B,high school,free/reduced,completed,76.0,85.0,82.0
342,group B,high school,standard,completed,69.0,76.0,74.0
534,group B,high school,standard,completed,73.0,69.0,68.0
66,group D,some high school,free/reduced,none,45.0,37.0,37.0
256,group C,associate's degree,free/reduced,none,64.0,73.0,68.0


In [413]:
y.sample(5)

676    female
310    female
280      male
308    female
216    female
Name: gender, dtype: object

In [414]:
# Создаем словарь для сохранения информации о моделях
models_info = {}

---
## 2
Выведите основную статистическую информацию о данных. 

**Сделайте количественное описание датасета:**
- число строк (объектов)
- число столбцов (признаков)
- статистику по признакам
- количество классов (значений целевой переменной).

In [415]:
# Число строк и столбцов
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   race.ethnicity               1000 non-null   object 
 1   parental.level.of.education  1000 non-null   object 
 2   lunch                        1000 non-null   object 
 3   test.preparation.course      1000 non-null   object 
 4   math.score                   1000 non-null   float64
 5   reading.score                1000 non-null   float64
 6   writing.score                1000 non-null   float64
dtypes: float64(3), object(4)
memory usage: 54.8+ KB


In [416]:
# Статистика по признакам
X.describe()

,math.score,reading.score,writing.score
count,1000.00000,1000.000000,1000.000000
mean,66.08900,69.169000,68.054000
std,15.16308,14.600192,15.195657
min,0.00000,17.000000,10.000000
25%,57.00000,59.000000,57.750000
50%,66.00000,70.000000,69.000000
75%,77.00000,79.000000,79.000000
max,100.00000,100.000000,100.000000


In [417]:
# Классы целевой переменной
print(*[f"{n+1}) {val}" for n, val in enumerate(set(y))], sep='\n')
print("\nВсего классов -", len(set(y)))

1) male
2) female

Всего классов - 2


---
## 3

Убедитесь, что данные пригодны для моделирования. 

В данных не должно быть пропущенных значений, все признаки должны быть численными.

Если эти условия нарушаются, исправьте это.

In [418]:
# Сначала заменим отсутствующие данные на средние
X = X.fillna(X.mean())

In [419]:
# Заменяем все не числовые данные на числовые
y = pd.factorize(y, sort=True)[0]

print("Заменяные столбцы:")
# Проходим по столбцам
for col_name in X.columns:
    # Если значения не числовые
    if not is_numeric_dtype(X[col_name].dtype):
        print(f"- {col_name}")
        # Заменяем на числовые
        X[col_name] = pd.factorize(X[col_name], sort=True)[0]


Заменяные столбцы:
- race.ethnicity
- parental.level.of.education
- lunch
- test.preparation.course


In [420]:
X.sample(5)

,race.ethnicity,parental.level.of.education,lunch,test.preparation.course,math.score,reading.score,writing.score
466,3,0,0,1,26.0,31.0,38.0
599,3,5,1,1,65.0,82.0,81.0
940,2,3,0,0,72.0,66.0,72.0
734,4,4,0,1,53.0,58.0,57.0
167,1,3,0,0,58.0,76.0,78.0


---
## 4

Обучите модель логистической регрессии на рассматриваемых данных из библиотеки `sklearn`.

Рассчитайте метрики accuracy и выведите таблицу классификации.


In [421]:
# Строим модель
logreg = LogisticRegression()
# Обучаем
interval = time()
logreg.fit(X, y)
interval = time() - interval

# Предсказываем новые данные
y_pred = logreg.predict(X)

In [422]:
ac_logistic = accuracy_score(y, y_pred)
print(f'accuracy = {ac_logistic:.3f}')
print('\nМатрица классификации:')
pd.DataFrame(confusion_matrix(y, y_pred))

accuracy = 0.90

Матрица классификации:


,0,1
0,470,48
1,57,425


In [423]:
# Сохраняем данные о модели
models_info["Логистическая регрессия"] = {
    "Время обучения": interval,
    "Точность": ac_logistic,
}

---
## 5
Обучите полиномиальную модель классификации.

Рассчитайте метрики accuracy и выведите таблицу классификации.

Попробуйте разные степени полинома и выберите ту, которая работает лучше.

In [424]:
ac_poly = 0

for n in range(9):
    # Создаем модель логистической регрессии
    logreg = LogisticRegression()

    # Объединяем ее с полиномиальной предобработкой
    X_poly = PolynomialFeatures(degree=n+1)
    model = make_pipeline(X_poly, logreg)

    # Обучаем модель
    interval = time()
    model.fit(X, y)
    interval = time() - interval
    # Предсказываем новые данные
    y_pred = model.predict(X)
    ac = accuracy_score(y, y_pred)
    if ac > ac_poly:
        ac_poly = ac
        inter = interval

    print('-'*20)
    print(f'Модель {n+1}-й степени полинома')
    print(f'accuracy = {ac:.3f}')
    print('\nМатрица классификации:')
    print(pd.DataFrame(confusion_matrix(y, y_pred)))


--------------------
Модель 1-й степени полинома
accuracy = 0.89

Матрица классификации:
     0    1
0  469   49
1   58  424
--------------------
Модель 2-й степени полинома
accuracy = 0.89

Матрица классификации:
     0    1
0  469   49
1   57  425
--------------------
Модель 3-й степени полинома
accuracy = 0.90

Матрица классификации:
     0    1
0  466   52
1   51  431
--------------------
Модель 4-й степени полинома
accuracy = 0.90

Матрица классификации:
     0    1
0  471   47
1   56  426
--------------------
Модель 5-й степени полинома
accuracy = 0.89

Матрица классификации:
     0    1
0  469   49
1   58  424
--------------------
Модель 6-й степени полинома
accuracy = 0.89

Матрица классификации:
     0    1
0  471   47
1   63  419
--------------------
Модель 7-й степени полинома
accuracy = 0.89

Матрица классификации:
     0    1
0  466   52
1   55  427
--------------------
Модель 8-й степени полинома
accuracy = 0.88

Матрица классификации:
     0    1
0  471   47
1   69  413


Результаты почти одинаковые, но наилучший у 3-й и 4-й степени полинома

In [425]:
# Сохраняем данные о модели
models_info["Полиномиальная регрессия"] = {
    "Время обучения": interval,
    "Точность": ac_poly,
}

---
## 6
Обучите модель классификации по методу опорных векторов.

Рассчитайте метрики accuracy и выведите таблицу классификации.

Попробуйте разные ядерные функции и выберите ту, которая работает лучше.


In [426]:
ac_svc = 0
for ker in ["linear", "poly", "rbf", "sigmoid"]:
    # Строим модель
    svm_linear = SVC(kernel=ker, max_iter=10_000)
    # Обучаем модель
    interval = time()
    svm_linear.fit(X, y)
    interval = time() - interval

    # Предсказываем новые данные
    y_pred = svm_linear.predict(X)

    ac = accuracy_score(y, y_pred)
    if ac > ac_poly:
        ac_svc = ac
        inter = interval

    ac_svc = max(ac_svc, ac)
    print('\n\n'+'-'*20)
    print('Ядро модели -', ker)
    print(f'accuracy = {ac:.3f}')
    print('\nМатрица классификации:')
    print(pd.DataFrame(confusion_matrix(y, y_pred)))




--------------------
Ядро модели - linear
accuracy = 0.90

Матрица классификации:
     0    1
0  469   49
1   56  426


--------------------
Ядро модели - poly
accuracy = 0.88

Матрица классификации:
     0    1
0  455   63
1   62  420


--------------------
Ядро модели - rbf
accuracy = 0.88

Матрица классификации:
     0    1
0  465   53
1   70  412


--------------------
Ядро модели - sigmoid
accuracy = 0.45

Матрица классификации:
     0    1
0  247  271
1  276  206


Лучше всего себя показала модель с линейным ядром

In [427]:
# Сохраняем данные о модели
models_info["Метод опорных векторов"] = {
    "Время обучения": interval,
    "Точность": ac_svc,
}

---
## 7

Обучите модель классификации Перцептрон.

Рассчитайте метрики accuracy и выведите таблицу классификации. 


In [428]:
# Создаем модель
clf = MLPClassifier()
# Обучаем
interval = time()
clf.fit(X, y)
interval = time() - interval

# Предсказываем новые данные
y_pred = clf.predict(X)

In [429]:
ac_perc = accuracy_score(y, y_pred)
print(f'accuracy = {ac_perc:.3f}')
print('\nМатрица классификации:')
pd.DataFrame(confusion_matrix(y, y_pred))

accuracy = 0.90

Матрица классификации:


,0,1
0,473,45
1,57,425


In [430]:
# Сохраняем данные о модели
models_info["Персептрон"] = {
    "Время обучения": interval,
    "Точность": ac_perc,
}

---
## 8
Выведите итоговую таблицу сравнения всех моделей.

В таблице должна быть информация о эффективности и времени обучения каждой модели.

Сделайте вывод о применимости и эффективности моделей для классификации объектов в данной задаче.


In [431]:
pd.DataFrame(models_info)

,Логистическая регрессия,Полиномиальная регрессия,Метод опорных векторов,Персептрон
Время обучения,0.018,1.846155,0.052537,1.040964
Точность,0.895,0.897000,0.895000,0.898000


На основе полученных результатов можно сделать вывод, что все модели хорошо себя показали, но самой быстрой оказалась логистическая регрессия, а самой точной - персептрон